In [1]:
from IPython import get_ipython

In [2]:
get_ipython().run_line_magic('tb', '')

No traceback available to show.


In [3]:
import sys
import argparse
from pathlib import Path
#Configuration file
config_file = 'preproc.args'
config_arg = '@'+config_file
print(sys.argv[0])
if config_arg not in sys.argv:
    primary = sys.argv[0]
    sys.argv = [primary,config_arg] 
print(sys.argv)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py
['/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py', '@preproc.args']


In [3]:
#from  src.run_preprocess import *
from src.run_debug import *

file = '@preproc.args'

print(file)
config = main(file)
pprint.pprint(config)

/opt/conda/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


@preproc.args
argv type:  <class 'list'>
argv:  ['/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py', '-f', '/home/jovyan/.local/share/jupyter/runtime/kernel-e0f006a9-566c-4e44-9887-8fbdf4ce46d5.json', '@preproc.args']


usage: ipykernel_launcher.py [-h] --problem_type PROBLEM_TYPE --output_file
                             OUTPUT_FILE --image_masks IMAGE_MASKS -o,
                             OUTPUT_DIR --all_modalities ALL_MODALITIES
                             --image_format IMAGE_FORMAT
                             [--input_images INPUT_IMAGES] [--CPU CPU]
                             [--patch_shape PATCH_SHAPE]
                             [--input_type INPUT_TYPE]
                             [--image_shape IMAGE_SHAPE]
                             [--overwrite OVERWRITE]
                             [--output_images OUTPUT_IMAGES]
                             [--output_imstats OUTPUT_IMSTATS]
                             [--training_modalities TRAINING_MODALITIES]
                             [--normalize NORMALIZE]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/jovyan/.local/share/jupyter/runtime/kernel-e0f006a9-566c-4e44-9887-8fbdf4ce46d5.json


SystemExit: 2

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [10]:
# Open the hdf5 file
if config['overwrite'] or not os.path.exists(config["output_file"]):
    hdf5_file = tables.open_file(config["output_file"], mode='w')
    overwrite = 1
else:
    hdf5_file = tables.open_file(config["output_file"], mode='r')

config["hdf5_file"] = hdf5_file

niftis = nifti_loader(
        config["hdf5_file"],
        config["input_images"],
        config["problem_type"],
        config["image_shape"],
        config["training_modalities"],
        config["image_masks"]
    )

print(' Image data shape: ', niftis.image_data_shape)
print(' Image modalities: ', niftis.image_modalities)
print('  No. of Subjects: ', len(niftis.ids))


 Image data shape:  (0, 1, 256, 256, None)
 Image modalities:  ['MR1']
  No. of Subjects:  9


In [12]:
niftis.hdf5.close()

In [13]:
niftis.image_data_shape = (0,1,256,256,32)

In [4]:
tables.Filters(5)

Filters(complevel=5, complib='zlib', shuffle=True, bitshuffle=False, fletcher32=False, least_significant_digit=None)

In [15]:
n_samples = len(niftis.ids)

#image_storage1 = niftis.hdf5.create_earray(niftis.hdf5.root, 'imdata', tables.Float32Atom(), 
#                                          shape=(0,1,256,256,32), filters=filters, expectedrows=n_samples)
image_storage2 = niftis.hdf5.create_vlarray(niftis.hdf5.root, 'imdata2', tables.Float32Atom(shape=()), filters=filters, expectedrows=n_samples)

AttributeError: 'File' object has no attribute 'root'

In [ ]:
from src.unet3d.normalize import reslice_image_set
for (i, imagePath) in enumerate(niftis.data_files):
    print(i)
    images, imstats = reslice_image_set(
                in_files=imagePath, image_shape=(256,256,None), label_indices=len(imagePath)-1, crop=True, stats=True)
    subject_data = [image.get_fdata() for image in images]
    break
    image_storage2.append(np.asarray(subject_data[:niftis.n_channels])[np.newaxis])
    print(imstats)
 

In [ ]:
subject_data[0].shape

In [ ]:
a=np.asarray([5,6])
a.shape

In [35]:
a = np.asarray(list(range(1,21))).reshape(2,2,5)
print(a.shape)
b = np.asarray(list(range(1,17))).reshape(2,2,4)
print(b.shape)

(2, 2, 5)
(2, 2, 4)


In [37]:
fileh.close()
filters = tables.Filters(complevel=5, complib='blosc')
n_samples=9
exp_slices = 5 # expected slices
fileh = tables.open_file('earrays.h5', mode='w')
vlarray = fileh.create_vlarray(fileh.root, 'vlarray1', tables.Int32Atom(shape=(1,2,2)),"ragged array of ints",
                              filters=filters)  #(1,2,2)


vlind = fileh.create_earray(fileh.root, 'vlind', tables.Int32Atom(), shape=(0,2), filters=filters, expectedrows=n_samples)


earray2 = fileh.create_earray(fileh.root, 'imdata_slices', tables.Int32Atom(), 
                                          shape=(0,1,2,2), filters=filters, expectedrows=n_samples*exp_slices)


#earray = fileh.create_earray(fileh.root, 'imdata', tables.Int32Atom(), 
#                                          shape=(0,1,2,2,5), filters=filters, expectedrows=n_samples)
start= 0
stop = 0

vlind.append(np.array([0,5])[np.newaxis])
vlind.append(np.array([5,10])[np.newaxis])

# Append some (variable length) rows:
vlarray.append(a.reshape(5,1,2,2))
vlarray.append(b.reshape(5,1,2,2))

earray.append(a[np.newaxis][np.newaxis])
earray.append(b[np.newaxis][np.newaxis])

earray2.append(a.reshape(5,1,2,2))
earray2.append(b.reshape(5,1,2,2))

# Now, read it through an iterator:
# print('-->', vlarray.title)
# for x in vlarray:
#     print('%s[%d]--> %s' % (vlarray.name, vlarray.nrow, x))

SyntaxError: invalid syntax (<ipython-input-37-f0daa10c3356>, line 26)

In [86]:
"/vlarray1" in fileh

True

In [24]:
earray[0]

array([[[[ 1,  2,  3,  4,  5],
         [ 6,  7,  8,  9, 10]],

        [[11, 12, 13, 14, 15],
         [16, 17, 18, 19, 20]]]], dtype=int32)

In [11]:
earray.shape

(2, 1, 2, 2, 5)

In [10]:
earray2.shape

(10, 1, 2, 2)

In [13]:
y = np.stack(earray2[vlind[0,0]:vlind[0,1]],axis=0)
y = y.reshape(1,2,2,5)
y.shape

(1, 2, 2, 5)

In [14]:
earray[0].shape

(1, 2, 2, 5)

In [22]:
earray[0].all() == y.all() == z.all()

True

In [21]:
z = vlarray[0].reshape(1,2,2,5)

In [26]:
image_data_shape = (0,1,2,2,None)
truth_data_shape = (0,1,2,2,None)
input_shape = (2,2,None)
if None in image_data_shape:
    Noneindex = input_shape.index(None)
    imarray_shape = tuple([i for i in image_data_shape if i])
    imtruth_shape = tuple([i for i in truth_data_shape if i])
    print(Noneindex)
    print(imarray_shape)
    print(imtruth_shape)

2
(1, 2, 2)
(1, 2, 2)


In [34]:
v = 10
a = 7
if v:
    a = 1
else:
    a = 2
    
print(a)    
    

1


In [38]:
len(None)

TypeError: object of type 'NoneType' has no len()